In [1]:
import numpy as np
import pandas as pd

from litedl.models import FeedForwardNeuralNetwork
from litedl.layers import Affine, Sigmoid, SoftmaxWithLoss
from litedl.optimizers import SGD
from litedl.utils import data_split, get_batches, OneHotEncoder, Standardizer

# 학습 하이퍼 파라미터 설정
epochs = 1000000
batch_size = 30

# 붓꽃 데이터셋 불러오기
dataset = pd.read_csv('dataset/Iris.csv')
dataset.drop('Id', axis=1, inplace=True)

# 피처와 라벨 데이터 나누기
features = dataset.iloc[:, :-1].values
labels = dataset.iloc[:, -1].values.reshape(-1, 1)

# 라벨 데이터를 원 핫 인코딩으로 표현하고 섞기
encoder = OneHotEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)
np.random.shuffle(labels)

# 피처 데이터를 표준화하기
standardizer = Standardizer()
standardizer.fit(features)
features = standardizer.transform(features)

# 학습용 데이터와 테스트용 데이터로 나누기
train_x, test_x = data_split(features, 0.5)
train_y, test_y = data_split(labels, 0.5)

# 미니배치 데이터 구하기
x_batches = get_batches(train_x, batch_size)
y_batches = get_batches(train_y, batch_size)

In [2]:
# 모델과 계층 선언
model = FeedForwardNeuralNetwork()
affine1 = Affine(input_size=4, output_size=5)
sigmoid = Sigmoid()
affine2 = Affine(input_size=5, output_size=3)
loss_fn = SoftmaxWithLoss()
optimizer = SGD()

# 계층을 쌓아 모델을 구축
model.add_layer(affine1)
model.add_layer(sigmoid)
model.add_layer(affine2)
model.add_loss_layer(loss_fn)

# 모델 학습
for epoch in range(epochs):
    for batch_index in range(len(x_batches)):
        loss = model.forward(x_batches[batch_index], y_batches[batch_index])
        model.backward()
        model.step(optimizer=optimizer)

model.to_pickle('classification.pkl')

In [3]:
# 테스트 데이터로 정확도 측정
pickle_model = FeedForwardNeuralNetwork.from_pickle('classification.pkl')

np.sum(np.argmax(pickle_model.predict(test_x), axis=1) == np.argmax(test_y, axis=1)) / test_x.shape[0]

np.float64(0.30666666666666664)